In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
import os

In [2]:
import logging
l = logging.getLogger(__name__)

In [3]:
logging.basicConfig(format='[%(asctime)s][%(levelname)s]: %(message)s',
                        level=logging.INFO)

In [4]:
# Preprocessing according to the pre-trained model
preprocess = transforms.Compose([
        transforms.Scale(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
           mean=[0.485, 0.456, 0.406],
           std=[0.229, 0.224, 0.225]
        ),
    ])

In [5]:
ds_train = datasets.ImageFolder("./transfer_learning/training", preprocess)

In [6]:
ds_train_ldr = torch.utils.data.DataLoader(ds_train, batch_size=1, shuffle=True)

In [7]:
ds_test = datasets.ImageFolder("./transfer_learning/testing", preprocess)

In [8]:
ds_test_ldr = torch.utils.data.DataLoader(ds_test, batch_size=1, shuffle=True)

In [21]:
ds_test.imgs

[('./transfer_learning/testing/coconut/coconut1.jpg', 0),
 ('./transfer_learning/testing/coconut/coconut2.jpg', 0),
 ('./transfer_learning/testing/coconut/coconut3.jpg', 0),
 ('./transfer_learning/testing/durian/durian1.png', 1),
 ('./transfer_learning/testing/durian/durian2.jpg', 1),
 ('./transfer_learning/testing/durian/durian3.jpg', 1),
 ('./transfer_learning/testing/lychee/lychee1.jpg', 2),
 ('./transfer_learning/testing/lychee/lychee2.jpg', 2),
 ('./transfer_learning/testing/lychee/lychee3.jpg', 2)]

In [9]:
def train(model, loss_function, optimizer, num_epochs=10):
    """
    Begin training the last layer
    :param model: Model to train
    :param loss_function: Use to assess the model
    :praram optimizer:
    :param num_epochs: Number of epochs to train on.
    :return: best model
    """
    
    
    best_model = model
    best_acc = 0
    
    for e in range(num_epochs):
        l.info("EPOCH: %d/%d", e, num_epochs-1)
        
        for is_training in [True, False]:
            model.train(is_training)
            
            if is_training:
                # Train the model
                for data, target in ds_train_ldr:
                    x, y = Variable(data), Variable(target)
                    
                    optimizer.zero_grad()
                    
                    r = model(x)
                    _, preds = torch.max(r.data, 1)
#                     l.info("Predictions: %s", str(preds))
                    
                    loss = loss_function(r, y)
#                     l.info("Loss: %s", str(loss))
                    
                    loss.backward()
                    optimizer.step()

            else:
                v_loss = 0
                v_acc = 0
                
                # Validate the model
                for data, target in ds_test_ldr:
                    x, y = Variable(data), Variable(target)
                    
                    optimizer.zero_grad()
                    
                    r = model(x)
                    _, preds = torch.max(r.data, 1)
#                     l.info("Predictions: %s", str(preds))
                    
                    loss = loss_function(r, y)
#                     l.info("Loss: %s", str(loss))
                    
                    v_loss += loss.data[0]
                    v_acc += torch.sum(preds == y.data)
                
                epoch_loss = v_loss / len(ds_test)
                epoch_acc = v_acc / len(ds_test)
                
                l.info("Epoch %d, loss %f, acc %f", e, epoch_loss, epoch_acc)
                
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model = copy.deepcopy(model)
    l.info("Complete. Best acc %f", best_acc)
    return best_model

In [10]:
model_conv = torchvision.models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

In [11]:
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 3)

In [12]:
loss_function = nn.CrossEntropyLoss()

In [13]:
optimizer = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

In [14]:
model_conv = train(
    model=model_conv, 
    loss_function=loss_function, 
    optimizer=optimizer, 
    num_epochs=10)

[2017-09-02 19:21:05,914][INFO]: EPOCH: 0/9
[2017-09-02 19:21:07,952][INFO]: Epoch 0, loss 1.144204, acc 0.444444
[2017-09-02 19:21:07,991][INFO]: EPOCH: 1/9
[2017-09-02 19:21:09,967][INFO]: Epoch 1, loss 1.267844, acc 0.333333
[2017-09-02 19:21:09,967][INFO]: EPOCH: 2/9
[2017-09-02 19:21:11,823][INFO]: Epoch 2, loss 1.182043, acc 0.333333
[2017-09-02 19:21:11,824][INFO]: EPOCH: 3/9
[2017-09-02 19:21:13,689][INFO]: Epoch 3, loss 1.029998, acc 0.333333
[2017-09-02 19:21:13,690][INFO]: EPOCH: 4/9
[2017-09-02 19:21:15,563][INFO]: Epoch 4, loss 1.022408, acc 0.444444
[2017-09-02 19:21:15,564][INFO]: EPOCH: 5/9
[2017-09-02 19:21:17,554][INFO]: Epoch 5, loss 1.145452, acc 0.333333
[2017-09-02 19:21:17,555][INFO]: EPOCH: 6/9
[2017-09-02 19:21:19,423][INFO]: Epoch 6, loss 0.997432, acc 0.444444
[2017-09-02 19:21:19,424][INFO]: EPOCH: 7/9
[2017-09-02 19:21:21,248][INFO]: Epoch 7, loss 1.051749, acc 0.444444
[2017-09-02 19:21:21,248][INFO]: EPOCH: 8/9
[2017-09-02 19:21:23,090][INFO]: Epoch 8, lo

In [15]:
def prepare_image(image):
    """
    Prepare image to be fed into the model.
    
    
    """
    
    # These preprocessing steps are taken from PyTorch's ImageNet example
    preprocess = transforms.Compose([
        transforms.Scale(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
           mean=[0.485, 0.456, 0.406],
           std=[0.229, 0.224, 0.225]
        ),
    ])
    return preprocess(image).unsqueeze_(0)

In [16]:
from PIL import Image

In [17]:
prepare_image(Image.open("durian2.jpg"))


( 0 , 0 ,.,.) = 
  2.2489  2.2489  2.2489  ...   2.2489  2.2489  2.2489
  2.2489  2.2489  2.2489  ...   2.2489  2.2489  2.2489
  2.2489  2.2489  2.2489  ...   2.2489  2.2489  2.2489
           ...             ⋱             ...          
  2.2489  2.2489  2.2489  ...   2.2489  2.2489  2.2489
  2.2489  2.2489  2.2489  ...   2.2489  2.2489  2.2489
  2.2489  2.2489  2.2489  ...   2.2489  2.2489  2.2489

( 0 , 1 ,.,.) = 
  2.4286  2.4286  2.4286  ...   2.4286  2.4286  2.4286
  2.4286  2.4286  2.4286  ...   2.4286  2.4286  2.4286
  2.4286  2.4286  2.4286  ...   2.4286  2.4286  2.4286
           ...             ⋱             ...          
  2.4286  2.4286  2.4286  ...   2.4286  2.4286  2.4286
  2.4286  2.4286  2.4286  ...   2.4286  2.4286  2.4286
  2.4286  2.4286  2.4286  ...   2.4286  2.4286  2.4286

( 0 , 2 ,.,.) = 
  2.6400  2.6400  2.6400  ...   2.6400  2.6400  2.6400
  2.6400  2.6400  2.6400  ...   2.6400  2.6400  2.6400
  2.6400  2.6400  2.6400  ...   2.6400  2.6400  2.6400
           

In [18]:
Variable(prepare_image(Image.open("durian2.jpg")))

Variable containing:
( 0 , 0 ,.,.) = 
  2.2489  2.2489  2.2489  ...   2.2489  2.2489  2.2489
  2.2489  2.2489  2.2489  ...   2.2489  2.2489  2.2489
  2.2489  2.2489  2.2489  ...   2.2489  2.2489  2.2489
           ...             ⋱             ...          
  2.2489  2.2489  2.2489  ...   2.2489  2.2489  2.2489
  2.2489  2.2489  2.2489  ...   2.2489  2.2489  2.2489
  2.2489  2.2489  2.2489  ...   2.2489  2.2489  2.2489

( 0 , 1 ,.,.) = 
  2.4286  2.4286  2.4286  ...   2.4286  2.4286  2.4286
  2.4286  2.4286  2.4286  ...   2.4286  2.4286  2.4286
  2.4286  2.4286  2.4286  ...   2.4286  2.4286  2.4286
           ...             ⋱             ...          
  2.4286  2.4286  2.4286  ...   2.4286  2.4286  2.4286
  2.4286  2.4286  2.4286  ...   2.4286  2.4286  2.4286
  2.4286  2.4286  2.4286  ...   2.4286  2.4286  2.4286

( 0 , 2 ,.,.) = 
  2.6400  2.6400  2.6400  ...   2.6400  2.6400  2.6400
  2.6400  2.6400  2.6400  ...   2.6400  2.6400  2.6400
  2.6400  2.6400  2.6400  ...   2.6400  2.6400

In [19]:
model_conv(Out[18])

Variable containing:
 0.3694  0.5323  0.2719
[torch.FloatTensor of size 1x3]